In [1]:
import numpy as np
from scipy.spatial.transform import Rotation as R
import math
import matplotlib.pyplot as plt
import argparse
import re
from scipy.spatial.transform import Rotation as R
import pandas as pd
import matplotlib.pyplot as plt
import os
from scipy.linalg import logm
from pyquaternion import Quaternion
from typing import Tuple, List, Dict
from tabulate import tabulate

In [2]:

def quaternion_inverse(given_quat):
    """
    Input: scipy format - scalar LAST. numpy.array([b, c, d, a]) where quaternion is a + bi + cj + dk
    or in wxyz format, w is scalar below
    Output: numpy array in same format but inverse of input
    """
    #pyquaternion library: Scalar FIRST: numpy.array([a, b, c, d]) where quaternion is a + bi + cj + dk
    qx, qy, qz, qw = given_quat #scalar last
    given_quat_scalar_first = np.array([qw, qx, qy, qz]) #scalar first
    quat_obj = Quaternion(array=given_quat_scalar_first) 
    inv_quat = quat_obj.inverse
    inv_quat_np = inv_quat.elements

    iqw, iqx, iqy, iqz  = inv_quat_np #scalar first
    inv_quat_np_scalar_first = np.array([iqx, iqy, iqz, iqw]) #scalar  last
    return  inv_quat_np_scalar_first


def get_rotation_error_using_arccos_of_trace_of_rotation(R_pred: np.ndarray, R_gt: np.ndarray) -> float:
    # See https://www.notion.so/saishubodh/Evaluation-Metrics-for-R-t-with-GT-Rotation-Translation-error-with-Ground-truth-476db686933048d5b74de36b382e0eec
        # measure the angular distance between two rotation matrice
    # R1,R2: [n, 3, 3]
    # Returns angle in degrees
    if R_pred.shape == (3,3):
        R_pred = R_pred[np.newaxis,:]
    if R_gt.shape == (3,3):
        R_gt = R_gt[np.newaxis,:]
    n = R_gt.shape[0]
    trace_idx = [0,4,8]
    trace = np.matmul(R_pred, R_gt.transpose(0,2,1)).reshape(n,-1)[:,trace_idx].sum(1)
    metric_in_degrees = np.arccos(((trace - 1)/2).clip(-1,1)) / np.pi * 180.0
    return metric_in_degrees[0]


def get_rotation_error_using_quaternion_dot(R_pred: np.ndarray, R_gt: np.ndarray) -> float:
    # See https://www.notion.so/saishubodh/Evaluation-Metrics-for-R-t-with-GT-Rotation-Translation-error-with-Ground-truth-476db686933048d5b74de36b382e0eec
    # cpp
    # const float d1 = std::fabs(q1.dot(q2));
    #d2 = std::fmin(1.0f, std::fmax(-1.0f, d1))
    #return 2 * acos(d2) * 180 / M_PI

    # return np.linalg.norm(logm(np.matmul(R_pred, R_gt.transpose()))) / np.sqrt(2)
    if R_pred.shape != (3, 3) or R_gt.shape != (3, 3):
        raise ValueError(f'Input matrices must be 3x3, instead got {R_pred.shape} and {R_gt.shape}')

    R_pred_obj, R_gt_obj = R.from_matrix(R_pred), R.from_matrix(R_gt)
    quat_pred, quat_gt = R_pred_obj.as_quat(), R_gt_obj.as_quat()

    # quat_pred_inv, quat_gt_inv = quaternion_inverse(quat_pred), quaternion_inverse(quat_gt)
    # quat_pred_inv_i, quat_gt_inv_i = quaternion_inverse(quat_pred_inv), quaternion_inverse(quat_gt_inv)
    # print(quat_pred, quat_gt)
    # print(quat_pred_inv, quat_gt_inv)
    # print(quat_pred_inv_i, quat_gt_inv_i)

    # d1 = abs(np.dot(quat_pred, quat_gt))
    # print("Quat inverse")
    d1 = abs(np.dot(quaternion_inverse(quat_pred), quat_gt))
    d2 = np.min((1.0, np.max((-1.0, d1))))
    rotation_error_in_degrees = 2 * np.arccos(d2) * 180 / np.pi
    # print(rotation_error_in_degrees)
    return rotation_error_in_degrees


In [3]:

def get_rotation_error_using_log_of_rotation(R_pred: np.ndarray, R_gt: np.ndarray) -> float:
    # See https://www.notion.so/saishubodh/Evaluation-Metrics-for-R-t-with-GT-Rotation-Translation-error-with-Ground-truth-476db686933048d5b74de36b382e0eec
    if R_pred.shape != (3, 3) or R_gt.shape != (3, 3):
        raise ValueError(f'Input matrices must be 3x3, instead got {R_pred.shape} and {R_gt.shape}')

    return np.linalg.norm(logm(np.matmul(R_pred, R_gt.transpose()))) / np.sqrt(2)

def get_translation_error(t_pred: np.ndarray, t_gt: np.ndarray) -> float:
    if t_gt.shape != (3, ) or t_gt.shape != (3, ):
        raise ValueError(f'Input vectors must be 3 dimensional, instead got {t_pred.shape} and {t_gt.shape}')

    return np.linalg.norm(t_gt - t_pred)

def get_both_errors(T_pred: np.ndarray, T_gt: np.ndarray) -> Tuple[float, float]:
    if T_pred.shape != (4, 4) or T_gt.shape != (4, 4):
        raise ValueError(f'Input matrices must be 4x4, instead got {T_pred.shape} and {T_gt.shape}')

    # rot_error = get_rotation_error_using_quaternion_dot(T_pred[:3, :3], T_gt[:3, :3])
    # print(rot_error, "quat")
    rot_error = get_rotation_error_using_log_of_rotation(T_pred[:3, :3], T_gt[:3, :3])
    # print(rot_error, "trace")
    trans_error = get_translation_error(T_pred[:3, -1], T_gt[:3, -1])
    return (rot_error, trans_error)

def get_errors(preds: List[np.ndarray], truths: List[np.ndarray]) -> Tuple[List[float], List[float]]:
    if len(preds) != len(truths):
        raise ValueError(f'Input must consist of equal numbers of predictions and ground truths, instead got {len(preds)} and {len(truths)}')
    
    rot_errors = []
    trans_errors = []

    for T_pred, T_gt in zip(preds, truths):
        rot_err, trans_err = get_both_errors(T_pred, T_gt)
        rot_errors.append(rot_err)
        trans_errors.append(trans_err)

    return rot_errors, trans_errors


In [4]:
def read_our_lp(file_path, freq):
	with open(file_path, "r") as f:
		corr = f.readlines()
	corr = [x.strip() for x in corr]
	corr = sorted(corr, key=lambda x: float(x.split(" ")[2]))
	new_corr = []
	for c in corr:
		c=c.strip()
		[l1, l2, s] = c.split()
		new_corr.append([
			freq*int(l1)+1,
			freq*int(l2)+1,
			float(s)
		])
	print(len(new_corr), len(new_corr))
	return new_corr

def read_rtabmap_lp(file_path):
	with open(file_path, "r") as f:
		corr = f.readlines()
	new_corr = []
	for c in corr:
		c=c.strip()
		[l1, l2] = c.split()
		new_corr.append([int(l1), int(l2)])
	
	print(len(new_corr), len(new_corr))
	return new_corr

def read_poses(file_path):
	with open(file_path, "r") as f:
		poses = f.readlines()
	poses = [x.strip() for x in poses]
	return poses

In [5]:
Our_LP = read_our_lp("LP/ENV2/qsmall/our.txt",3)
Poses = read_poses("/home/ayushsharma/Documents/College/RRC/IROS2023/habitat-data/test/ENV2/qsmall/poses.csv")
Rtabmap_LP = read_rtabmap_lp("LP/ENV2/qsmall/rtabmap_qsmall_lp.txt")

55 55
32 32


In [10]:
def get_err(Our_LP, Poses, flag=True):
    ourLPErr = []
    for lp in Our_LP:
        if flag:
            [l1, l2, _] = lp
        else:
            [l1, l2] = lp
        p1 = Poses[l1].strip().split(',')
        p2 = Poses[l2].strip().split(',')
        q1 = [
            float(p1[4]),
            float(p1[5]),
            float(p1[6]),
            float(p1[7]),
        ]
        q2 = [
            float(p2[4]),
            float(p2[5]),
            float(p2[6]),
            float(p2[7]),
        ]
        r1 = R.from_quat(q1).as_matrix()
        r2 = R.from_quat(q2).as_matrix()
        # T1 = np.zeros((4,4))
        # T2 = np.zeros((4,4))
        # T1[:3,:3] = r1
        # T2[:3,:3] = r2
        # T1[:,3] = p1[1:4]
        # T2[:,3] = p2[1:4]
        ourLPErr.append(get_rotation_error_using_quaternion_dot(r1, r2))
    return ourLPErr

In [11]:
ourLPErr = get_err(Our_LP, Poses)
rtabmapLPErr = get_err(Rtabmap_LP, Poses, False)

In [12]:
for id,e in enumerate(ourLPErr):
    print(Our_LP[id], e)
print("rtabmap")
for id,e in enumerate(rtabmapLPErr):
    print(Rtabmap_LP[id], e)

[148, 157, 0.8006738424301147] 19.999999759894255
[187, 754, 0.8018497228622437] 19.999964626492243
[493, 661, 0.8019539713859558] 160.00003284860756
[499, 667, 0.8025699257850647] 160.00003284860756
[673, 685, 0.8033798336982727] 0.0
[340, 334, 0.8042711019515991] 0.0
[469, 790, 0.8050551414489746] 169.99996509256738
[217, 211, 0.8053633570671082] 0.0
[454, 457, 0.8055416345596313] 10.000001099841285
[670, 679, 0.8068163394927979] 0.0
[472, 790, 0.806876540184021] 169.99996509256738
[499, 664, 0.8070405721664429] 160.00003284860756
[496, 673, 0.807561457157135] 160.00003284860756
[502, 661, 0.8078940510749817] 170.000031203606
[751, 181, 0.8095076084136963] 19.999964626492243
[490, 487, 0.8095747828483582] 19.99999858932556
[499, 673, 0.8106867074966431] 160.00003284860756
[466, 778, 0.8111031651496887] 169.99996509256738
[496, 664, 0.8118925094604492] 160.00003284860756
[145, 157, 0.8121135234832764] 19.999999759894255
[496, 667, 0.812843918800354] 160.00003284860756
[493, 664, 0.814

In [ ]:


def print_statistics(data: List[Dict]) -> None:
    table = []
    for dataset in data:
        res = {
            'Dataset': dataset['name']+":mean",
            'Rot. Error': np.mean(dataset['rot_errors']),
            'Trans. Error': np.mean(dataset['trans_errors']),
        }
        table.append(res)
        res = {
            'Dataset': dataset['name']+":max",
            'Rot. Error': np.max(dataset['rot_errors']),
            'Trans. Error': np.max(dataset['trans_errors']),
        }
        table.append(res)
        res = {
            'Dataset': dataset['name']+":min",
            'Rot. Error': np.min(dataset['rot_errors']),
            'Trans. Error': np.min(dataset['trans_errors']),
        }
        table.append(res)

    # print results
    print(tabulate(table, headers='keys', tablefmt='presto'))

    # histogram
    fig, axs = plt.subplots(len(data), 2)
    axs = axs.reshape((len(data), 2)) # when len(data) = 1
    for i in range(len(data)):
        r = data[i]['rot_errors']
        t = data[i]['trans_errors']
        axs[i, 0].hist(r, weights=np.ones(len(r))/len(r))
        axs[i, 0].set_title(data[i]['name'])
        axs[i, 0].set(xlabel='Rot. Error', ylabel='Fraction')

        axs[i, 1].hist(t, weights=np.ones(len(t))/len(t))
        axs[i, 1].set_title(data[i]['name'])
        axs[i, 1].set(xlabel='Trans. Error', ylabel='Fraction')

    plt.tight_layout()
    plt.show()




In [ ]:


parser = argparse.ArgumentParser(description='PLOTTING THE RETRIEVAL')
parser.add_argument('--root_dir', type=str, default='', help='Path to dataset')
parser.add_argument('--dataset', type=str, default='berlin', 
        help='Dataset to use', choices=['spair_90', 'spair_180','SMALL','TOPVIEW','PERSPECTIVE','sT4fr6TAbpF','oxford', 'nordland', 'berlin'])
parser.add_argument('--netvlad_predictions', type=str, default='', help='Path to NetVLAD Predictions')
parser.add_argument('--reference_poses', type=str, default='', help='Path to reference_poses that needs to be filtered')
parser.add_argument('--query_poses', type=str, default='', help='Path to query_poses that needs to be filtered')
parser.add_argument('--rord_trans', type=str, default='', help='Path to predicted transformation computed using RoRD')
parser.add_argument('--name', type=str, default='', help='Name to plot')


if __name__ == "__main__":
    opt = parser.parse_args()
    dataset = get_whole_val_set(opt.root_dir, opt.dataset.lower())

    db_lst = dataset.dbStruct.dbImage
    db_lst = np.array([int(re.search('\d+', x.replace(' ',''))[0]) for x in db_lst])
    q_lst = dataset.dbStruct.qImage
    q_lst = np.array([int(re.search('\d+', x.replace(' ',''))[0]) for x in q_lst])
    db_cood_lst = dataset.dbStruct.locDb
    q_cood_lst = dataset.dbStruct.locQ
    X_Db, Y_Db = db_cood_lst[:,0], db_cood_lst[:,1]
    X_Q, Y_Q = q_cood_lst[:,0], q_cood_lst[:,1]

    # since we didn't included Rot. in mat files that's why we need to do this.

    df = pd.read_csv(opt.reference_poses)
    col = df.columns
    db_lst = np.array(db_lst)-1
    [ db_qw, db_qx, db_qz, db_qy] = [df[col[4]][db_lst], df[col[5]][db_lst], df[col[6]][db_lst], df[col[7]][db_lst]]
    quat_Db = np.vstack((db_qx, db_qy,db_qz, db_qw)).T
    Rot_Db = R.from_quat(quat_Db).as_matrix()
    Transform_Db = np.zeros((Rot_Db.shape[0],4,4))
    Transform_Db[:,:3,:3] = Rot_Db
    Transform_Db[:,0,3] = X_Db
    Transform_Db[:,1,3] = Y_Db
    Transform_Db[:,3,3] = np.ones(Rot_Db.shape[0])

    df = pd.read_csv(opt.query_poses)
    col = df.columns
    q_lst = np.array(q_lst)-1
    [ q_qw, q_qx, q_qz, q_qy] = [df[col[4]][q_lst], df[col[5]][q_lst], df[col[6]][q_lst], df[col[7]][q_lst]]
    quat_Q = np.vstack((q_qx, q_qy,q_qz, q_qw)).T
    Rot_Q = R.from_quat(quat_Q).as_matrix()
    Transform_Q = np.zeros((Rot_Q.shape[0],4,4))
    Transform_Q[:,:3,:3] = Rot_Q
    Transform_Q[:,0,3] = X_Q
    Transform_Q[:,1,3] = Y_Q
    Transform_Q[:,3,3] = np.ones(Rot_Q.shape[0])


    data = []
    preds = []
    truths = []

    netvlad_candidates = np.load(opt.netvlad_predictions)
    for i in range(netvlad_candidates.shape[0]):
        for j in range(1):
            u = np.argwhere(q_lst==q_lst[i])[0][0]
            v = np.argwhere(db_lst==db_lst[int(netvlad_candidates[i,j])])[0][0]
            R_gt = np.linalg.inv(Transform_Q[u]) @ Transform_Db[v]
            R_pred = np.load(os.path.join(opt.rord_trans,str(q_lst[i]+1)+"-"+str(1+db_lst[int(netvlad_candidates[i,j])])+".npy"))
            R_pred[2,3] = 0
            truths.append(R_gt) 
            preds.append(R_pred) 
            if(i==0):
                print(R_gt,  "\n\n\n")
                print(R_pred)

    rot_errors, trans_errors = get_errors(preds, truths)
    data.append({
        'name': 'Query-Retrieved ('+ opt.name + ")",
        'rot_errors': rot_errors,
        'trans_errors': trans_errors,
    })
    print_statistics(data)   

In [ ]:

def transformation_matrix_from_7(x1, y1, z1, qx1, qy1, qz1, qw1):
	r1 = R.from_quat([qx1, qy1, qz1, qw1])
	m1 = np.hstack((r1.as_matrix(), np.array([x1,y1,z1], dtype=np.float64).reshape(3,1)))
	m1 = np.vstack((m1, np.array([0,0,0,1], dtype=np.float64).reshape(1,4)))
	return m1

def quat2eulers(qw,qx,qy, qz):
	
	r = R.from_quat([qx, qy, qz, qw])
	roll, pitch, yaw = r.as_euler('xyz', degrees=True)
	return roll, pitch, yaw	


In [ ]:

with open("LP/ENV2/qsmall/our.txt", "r") as f:
	corr = f.readlines()
corr = [x.strip() for x in corr]
corr = sorted(corr, key=lambda x: float(x.split(" ")[2]))
corr = corr[-32:]
print(len(corr))

with open("/home/ayushsharma/Documents/College/RRC/IROS2023/habitat-data/test/ENV2/qsmall/poses.csv", "r") as f:
	poses = f.readlines()
poses = [x.strip() for x in poses]
	

In [ ]:
for t in corr[-32:]:
    print(t)

In [ ]:

pitchlist = []
translationlist = []
skipped = 0
print("l1,l2,r,p,y,d")
for line in corr:
	l1 = int(line.split(" ")[0])
	l2 = int(line.split(" ")[1])
	_ , x1, y1, z1, qw1, qx1, qy1, qz1 = poses[3*l1 + 1].split(",")
	_ , x2, y2, z2, qw2, qx2, qy2, qz2 = poses[3*l2 + 1].split(",")
	r1, p1, yaw1 = quat2eulers(float(qw1), float(qx1), float(qy1), float(qz1))
	r2, p2, yaw2 = quat2eulers(float(qw2), float(qx2), float(qy2), float(qz2))
	x1 = float(x1)
	x2 = float(x2)
	y1 = float(y1)
	y2 = float(y2)
	z1 = float(z1)
	z2 = float(z2)
	# print(x1, x2, y1, y`2, z1, z2)
	# print(f"{l1},{l2},{r1-r2},{p1-p2},{y1-y2},{math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)}")
	delta_translation = math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
	if(delta_translation > 3):
		# print(f"skipped {l1} {l2} because of large translation difference {delta_translation}")
		# skipped +=1
		print("dt ",3*l1+1, 3*l2+1, delta_translation, (r1,r2), (p1,p2), (yaw1,yaw2))
	pitchlist.append(abs(p1-p2))
	if abs(p1-p2)>60:
		print(3*l1+1, 3*l2+1, p1, p2)
	translationlist.append(math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))


In [ ]:
-0.41388404	0.056315005	-13.540111	0.996192574501038	0	0.0871804431080818	0
-0.6029608	0.056315005	-16.773697	2.50637531280518E-05	0	-1	0


In [ ]:
# fig = plt.figure(figsize=(10,5))
fig, ax = plt.subplots(2,2, figsize=(10,10))

ax[0,0].hist(pitchlist, bins=range(0,181, 10), density=True)
ax[0,0].set_title("angle difference, ours ")
# plt.show()
ax[0, 1].hist(translationlist, bins=range(0,21,2), density=True)
ax[0, 1].set_title("translation difference, ours")

with open("LP/ENV2/qsmall/rtabmap_qsmall_lp.txt", "r") as f:
	corr = f.readlines()
corr = [x.strip() for x in corr]
for t in corr:
	print(t)

pitchlist = []
translationlist = []
skipped = 0
print("l1,l2,r,p,y,d")
for line in corr:
	l1 = int(line.split(" ")[0])
	l2 = int(line.split(" ")[1])
	_ , x1, y1, z1, qw1, qx1, qy1, qz1 = poses[l1].split(",")
	_ , x2, y2, z2, qw2, qx2, qy2, qz2 = poses[l2].split(",")
	r1, p1, yaw1 = quat2eulers(float(qw1), float(qx1), float(qy1), float(qz1))
	r2, p2, yaw2 = quat2eulers(float(qw2), float(qx2), float(qy2), float(qz2))
	x1 = float(x1)
	x2 = float(x2)
	y1 = float(y1)
	y2 = float(y2)
	z1 = float(z1)
	z2 = float(z2)
	# print(x1, x2, y1, y`2, z1, z2)
	# print(f"{l1},{l2},{r1-r2},{p1-p2},{y1-y2},{math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)}")
	delta_translation = math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
	# if(delta_translation > 5):
	# 	print(f"skipped {l1} {l2} because of large translation difference {x1} {x2} {y1} {y2} {z1} {z2}")
	# 	skipped +=1
	# 	continue
	pitchlist.append(abs(p1-p2))
	translationlist.append(math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2))
ax[1,0].hist(pitchlist, bins=range(0,181,10), density=True)
ax[1,0].set_title("angle difference, rtabmap")
# plt.show()
ax[1, 1].hist(translationlist, bins=range(0,21,2), density=True)
ax[1, 1].set_title("translation difference, rtabmap")
plt.show()